In [ ]:
! pip install scikit-learn

In [13]:
! pip install keras

     |████████████████████████████████| 378kB 155kB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/28/6a/8c1f62c37212d9fc441a7e26736df51ce6f0e38455816445471f10da4f0a/Keras_Preprocessing-1.1.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/71/e3/19762fdfc62877ae9102edf6342d71b28fbfd9dea3d2f96a882ce099b03f/Keras_Applications-1.0.8-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/10/56/d5c53cd170529bb40cd7dd43e2b68944cb65a45f65ab4c78a68f4ac9e51e/h5py-2.10.0-cp35-cp35m-manylinux1_x86_64.whl


In [14]:
! pip install tensorflow

     |████████████████████████████████| 421.8MB 18kB/s  eta 0:00:01     |██████████████                  | 183.4MB 4.5MB/s eta 0:00:54     |████████████████████▍           | 269.2MB 35kB/s eta 1:12:24kB/s eta 0:54:47��███▏          | 279.3MB 75kB/s eta 0:31:17��███▎          | 280.4MB 27kB/s eta 1:26:04��███▎          | 280.8MB 60kB/s eta 0:38:41     |██████████████████████▎         | 294.0MB 116kB/s eta 0:18:14████████████████▌         | 296.8MB 78kB/s eta 0:26:28     |██████████████████████▉         | 300.7MB 143kB/s eta 0:14:02█████████████████         | 302.0MB 64kB/s eta 0:30:57 eta 0:40:11��▉       | 327.8MB 49kB/s eta 0:31:33��█████████████████▎      | 333.1MB 145kB/s eta 0:10:13   | 359.8MB 140kB/s eta 0:07:23     |███████████████████████████▋    | 364.3MB 147kB/s eta 0:06:31   | 369.0MB 145kB/s eta 0:06:02��███████████████▋  | 389.6MB 76kB/s eta 0:07:01��████████████████████▉  | 392.7MB 130kB/s eta 0:03:43ta 0:01:10     |████████████████████████████████| 420.4MB 615kB/s eta 0:

     |████████████████████████████████| 153kB 65kB/s  eta 0:00:01
  Created wheel for opt-einsum: filename=opt_einsum-3.1.0-cp35-none-any.whl size=61682 sha256=5607c20d12ca383205bfb6eb06b1e609901c1af2c1c6decc69995357a87c888a
  Stored in directory: /home/noomkcalb/.cache/pip/wheels/2c/b1/94/43d03e130b929aae7ba3f8d15cbd7bc0d1cb5bb38a5c721833
  Created wheel for absl-py: filename=absl_py-0.9.0-cp35-none-any.whl size=121932 sha256=327990282fc1332ff0d5beab2b59327a1f9772a6da760c3e5a9e91215f5c3230
  Stored in directory: /home/noomkcalb/.cache/pip/wheels/8e/28/49/fad4e7f0b9a1227708cbbee4487ac8558a7334849cb81c813d
Successfully built opt-einsum absl-py


In [201]:
! pip install tqdm

     |████████████████████████████████| 61kB 31kB/s eta 0:00:01


In [1]:
import pandas as pd

In [2]:
import mne

In [3]:
import numpy as np

In [4]:
from tqdm import tqdm_notebook

In [5]:
import matplotlib.pyplot as plt

In [6]:
from keras.models import Sequential
from keras.layers import Conv1D, MaxPool1D, Flatten, Dropout, Dense, Conv2D, MaxPool2D

Using TensorFlow backend.


In [7]:
from keras.utils import to_categorical

In [8]:
SAMPLE_RATE = 250
WINDOW_DURATION = 2
CHANNELS = 3
ALL_FREQUENCIES = np.arange(7, 45, 1)
FREQUENCIES = ALL_FREQUENCIES.shape[0]

- Input - 2s spectre
- Output - left, right, none

# Dataset extraction

Подготовка датасета:
- Для каждого файла, соответствующего таске 1:
    - Считываем
    - Получаем tfr для каждого события
    - Нормируем tfr к первым секундам
    - Выбираем каналы
    - Сохраняем в X дату
- Меняем формат на channels_first
- При разделении датасета:
    - Используем ридер, для которого указываем субъектов для считывания
    - Делаем рандомный селект субъектов

Не забыть активировать видеокарту!

In [9]:
def read_epochs(path, tmin=-1, tmax=4):
    eeg = mne.io.read_raw_edf(
        path, 
        preload=True,
        exclude=["T9..", "T10."]
    )
    eeg.rename_channels(lambda x: x.strip('.'))
    eeg.filter(2, 50)
    events, annot_to_events = mne.events_from_annotations(eeg)
    picks = mne.pick_channels(eeg.info["ch_names"], ["C3", "Cz", "C4"])
    epochs = mne.Epochs(
        eeg, 
        events, 
        event_id={
            "none": 1,
            "left": 2,
            "right": 3
        },
        tmin=tmin - 0.5,
        tmax=tmax + 0.5,
        picks=picks,
        preload=True,
    )
    return epochs

In [10]:
epochs = read_epochs("./S001R03.edf")

Extracting EDF parameters from /home/noomkcalb/Документы/neuroscience/right-left/S001R03.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 2 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 2.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 1.00 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 265 samples (1.656 sec)

Used Annotations descriptions: ['T2', 'T1', 'T0']
30 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated
Loading data for 30 events and 961 

In [11]:
def get_power(epochs, event, baseline, tmin=-1, tmax=4, frequencies=ALL_FREQUENCIES):
    power = mne.time_frequency.tfr_multitaper(
        epochs[event], 
        freqs=frequencies, 
        n_cycles=frequencies,
        return_itc=False, 
        decim=3, 
        n_jobs=1,
        average=False
    )
    power.apply_baseline(baseline, mode='percent')
    power.crop(tmin, tmax)
    return power.data

In [12]:
power = get_power(epochs, "left", [-1, 0])

Not setting metadata
Applying baseline correction (mode: percent)


In [15]:
def read_data(subjects):
    X = []
    y = []
    for subject in tqdm_notebook(subjects):
        path = "./full-data/eegmmidb-1.0.0.physionet.org/S{:03d}/".format(subject)
        for task in [3, 7, 11]:
            file = "S{:03d}R{:02d}.edf".format(subject, task)
            full_path = path + file
            print("Reading:", full_path)
            try:
                epochs = read_epochs(full_path)
            except:
                print("File not exists")
                continue
            for event_index, event in enumerate(["none", "left", "right"]):
                if event == "none":
                    baseline = [2, 3]
                else:
                    baseline = [-1, 0]
                power = get_power(epochs, event, baseline)
                X.append(power)
                y += [event_index] * power.shape[0]
    return X, to_categorical(y)

In [14]:
X, y = read_data([1])

/home/noomkcalb/.virtualenvs/neuro/lib/python3.5/site-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


Reading: ./full-data/eegmmidb-1.0.0.physionet.org/S001/S001R03.edf
Extracting EDF parameters from /home/noomkcalb/Документы/neuroscience/right-left/full-data/eegmmidb-1.0.0.physionet.org/S001/S001R03.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 2 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 2.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 1.00 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 265 samples (1.656 sec)

Used Annotations descriptions: ['T2', 'T1', 'T0']
30 matching events found
Applying baselin

In [16]:
train_rate = 0.7
all_subjects = np.arange(1, 109)
train_subjects = np.random.choice(all_subjects, int(train_rate * len(all_subjects)))
test_subjects = [s for s in all_subjects if s not in train_subjects]

In [17]:
train_X, train_y = read_data(train_subjects)

/home/noomkcalb/.virtualenvs/neuro/lib/python3.5/site-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


Reading: ./full-data/eegmmidb-1.0.0.physionet.org/S041/S041R03.edf
Extracting EDF parameters from /home/noomkcalb/Документы/neuroscience/right-left/full-data/eegmmidb-1.0.0.physionet.org/S041/S041R03.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19679  =      0.000 ...   122.994 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 2 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 2.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 1.00 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 265 samples (1.656 sec)

Used Annotations descriptions: ['T2', 'T1', 'T0']
30 matching events found
Applying baselin

EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 2 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 2.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 1.00 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 265 samples (1.656 sec)

Used Annotations descriptions: ['T2', 'T1', 'T0']
30 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated
Loading data for 30 events and 961 original time points ...
1 bad epochs dropped
Not setting metadata
Applying baseline correction 

Reading 0 ... 19679  =      0.000 ...   122.994 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 2 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 2.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 1.00 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 265 samples (1.656 sec)

Used Annotations descriptions: ['T2', 'T1', 'T0']
30 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated
Loading data for 30 events and 961 original time points ...
2 bad epochs dropped
Not setting metadata
Applying baseline correction (mode: percent)
Not setting metadata
Applying baseline correction (mode: percent)
N

Setting up band-pass filter from 2 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 2.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 1.00 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 265 samples (1.656 sec)

Used Annotations descriptions: ['T2', 'T1', 'T0']
30 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated
Loading data for 30 events and 961 original time points ...
2 bad epochs dropped
Not setting metadata
Applying baseline correction (mode: percent)
Not setting metadata
Applying baseline correction (mode: percent)
Not setting metadata
Applying baseline correction (mode: percent)
Reading: ./full-data/eegmmidb-1.0.

...

- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 2.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 1.00 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 265 samples (1.656 sec)

Used Annotations descriptions: ['T2', 'T1', 'T0']
30 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated
Loading data for 30 events and 961 original time points ...
2 bad epochs dropped
Not setting metadata
Applying baseline correction (mode: percent)
Not setting metadata
Applying baseline correction (mode: percent)
Not setting metadata
Applying baseline correction (mode: percent)
Reading: ./full-data/eegmmidb-1.0.0.physionet.org/S071/S071R03.edf
Extracting EDF parameters from /home/noomkcalb/Документы/neuroscience/right-left/full-data/eegmmidb-1.0.0.physionet.o

- Lower passband edge: 2.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 1.00 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 265 samples (1.656 sec)

Used Annotations descriptions: ['T2', 'T1', 'T0']
30 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated
Loading data for 30 events and 961 original time points ...
1 bad epochs dropped
Not setting metadata
Applying baseline correction (mode: percent)
Not setting metadata
Applying baseline correction (mode: percent)
Not setting metadata
Applying baseline correction (mode: percent)
Reading: ./full-data/eegmmidb-1.0.0.physionet.org/S049/S049R03.edf
Extracting EDF parameters from /home/noomkcalb/Документы/neuroscience/right-left/full-data/eegmmidb-1.0.0.physionet.org/S049/S049R03.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19

- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 265 samples (1.656 sec)

Used Annotations descriptions: ['T2', 'T1', 'T0']
30 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated
Loading data for 30 events and 961 original time points ...
2 bad epochs dropped
Not setting metadata
Applying baseline correction (mode: percent)
Not setting metadata
Applying baseline correction (mode: percent)
Not setting metadata
Applying baseline correction (mode: percent)
Reading: ./full-data/eegmmidb-1.0.0.physionet.org/S097/S097R03.edf
Extracting EDF parameters from /home/noomkcalb/Документы/neuroscience/right-left/full-data/eegmmidb-1.0.0.physionet.org/S097/S097R03.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19679  =      0.000 ...   122.994 secs...
Filtering raw data in 1 contiguous segment
Setting up band-p

- Filter length: 265 samples (1.656 sec)

Used Annotations descriptions: ['T2', 'T1', 'T0']
30 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated
Loading data for 30 events and 961 original time points ...
2 bad epochs dropped
Not setting metadata
Applying baseline correction (mode: percent)
Not setting metadata
Applying baseline correction (mode: percent)
Not setting metadata
Applying baseline correction (mode: percent)
Reading: ./full-data/eegmmidb-1.0.0.physionet.org/S078/S078R03.edf
Extracting EDF parameters from /home/noomkcalb/Документы/neuroscience/right-left/full-data/eegmmidb-1.0.0.physionet.org/S078/S078R03.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19679  =      0.000 ...   122.994 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 2 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, n

30 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated
Loading data for 30 events and 961 original time points ...
1 bad epochs dropped
Not setting metadata
Applying baseline correction (mode: percent)
Not setting metadata
Applying baseline correction (mode: percent)
Not setting metadata
Applying baseline correction (mode: percent)



In [18]:
test_X, test_y = read_data(test_subjects)

/home/noomkcalb/.virtualenvs/neuro/lib/python3.5/site-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


Reading: ./full-data/eegmmidb-1.0.0.physionet.org/S001/S001R03.edf
Extracting EDF parameters from /home/noomkcalb/Документы/neuroscience/right-left/full-data/eegmmidb-1.0.0.physionet.org/S001/S001R03.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 2 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 2.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 1.00 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 265 samples (1.656 sec)

Used Annotations descriptions: ['T2', 'T1', 'T0']
30 matching events found
Applying baselin

EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19679  =      0.000 ...   122.994 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 2 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 2.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 1.00 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 265 samples (1.656 sec)

Used Annotations descriptions: ['T2', 'T1', 'T0']
30 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated
Loading data for 30 events and 961 original time points ...
2 bad epochs dropped
Not setting metadata
Applying baseline correction 

Reading 0 ... 19679  =      0.000 ...   122.994 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 2 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 2.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 1.00 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 265 samples (1.656 sec)

Used Annotations descriptions: ['T2', 'T1', 'T0']
30 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated
Loading data for 30 events and 961 original time points ...
2 bad epochs dropped
Not setting metadata
Applying baseline correction (mode: percent)
Not setting metadata
Applying baseline correction (mode: percent)
N

Setting up band-pass filter from 2 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 2.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 1.00 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 265 samples (1.656 sec)

Used Annotations descriptions: ['T2', 'T1', 'T0']
30 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated
Loading data for 30 events and 961 original time points ...
2 bad epochs dropped
Not setting metadata
Applying baseline correction (mode: percent)
Not setting metadata
Applying baseline correction (mode: percent)
Not setting metadata
Applying baseline correction (mode: percent)
Reading: ./full-data/eegmmidb-1.0.

...

Not setting metadata
Applying baseline correction (mode: percent)
Not setting metadata
Applying baseline correction (mode: percent)
Not setting metadata
Applying baseline correction (mode: percent)
Reading: ./full-data/eegmmidb-1.0.0.physionet.org/S100/S100R11.edf
Extracting EDF parameters from /home/noomkcalb/Документы/neuroscience/right-left/full-data/eegmmidb-1.0.0.physionet.org/S100/S100R11.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 15743  =      0.000 ...   122.992 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 2 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 2.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 1.00 Hz)
- Upper passband edge: 50.00 Hz
- Upper t

<ipython-input-9-32745a652d46>:5: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  exclude=["T9..", "T10."]


Not setting metadata
Applying baseline correction (mode: percent)
Not setting metadata
Applying baseline correction (mode: percent)
Not setting metadata
Applying baseline correction (mode: percent)
Reading: ./full-data/eegmmidb-1.0.0.physionet.org/S101/S101R03.edf
Extracting EDF parameters from /home/noomkcalb/Документы/neuroscience/right-left/full-data/eegmmidb-1.0.0.physionet.org/S101/S101R03.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 2 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 2.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 1.00 Hz)
- Upper passband edge: 50.00 Hz
- Upper t

Not setting metadata
Applying baseline correction (mode: percent)
Not setting metadata
Applying baseline correction (mode: percent)
Reading: ./full-data/eegmmidb-1.0.0.physionet.org/S105/S105R03.edf
Extracting EDF parameters from /home/noomkcalb/Документы/neuroscience/right-left/full-data/eegmmidb-1.0.0.physionet.org/S105/S105R03.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19679  =      0.000 ...   122.994 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 2 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 2.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 1.00 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
-

Not setting metadata
Applying baseline correction (mode: percent)
Reading: ./full-data/eegmmidb-1.0.0.physionet.org/S107/S107R03.edf
Extracting EDF parameters from /home/noomkcalb/Документы/neuroscience/right-left/full-data/eegmmidb-1.0.0.physionet.org/S107/S107R03.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 2 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 2.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 1.00 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 265 samples (1.656 sec)

Used Annotations descript

In [19]:
def process_signals(X, min_size=None):
    if not min_size:
        min_size = min([x.shape[-1] for x in X])
    stacked_X = np.vstack([x[..., :min_size] for x in X])
    return np.swapaxes(stacked_X, 1, -1)

In [20]:
prepared_train_X = process_signals(train_X)

In [21]:
SAMPLES = prepared_train_X.shape[1]

In [25]:
prepared_train_X.shape

(6396, 215, 38, 3)

In [22]:
prepared_test_X = process_signals(test_X, min_size=SAMPLES)

In [24]:
prepared_test_X.shape

(4845, 215, 38, 3)

# Model code

- Input channels x frequencies x time
- Conv Fx3, 8
- Maxpool
- Conv Fx3, 64
- Maxpool
- Flatten
- Dropout, 0.2
- Full, 100
- Dropout, 0.2
- Softmax, 3

In [264]:
# SAMPLES = SAMPLE_RATE * WINDOW_DURATION

In [26]:
CONVOLUTIONS = [8, 32, 64]
DROPOUT_RATE = 0.2
FULLY_CONNECTED = 100
BATCH_SIZE = 64

In [27]:
model = Sequential()

model.add(Conv2D(
    CONVOLUTIONS[0], (3, 3), 
    input_shape=(SAMPLES, FREQUENCIES, CHANNELS),
    padding='same'
))

for prev, filters in enumerate(CONVOLUTIONS[1:]):
    model.add(Conv2D(filters, (3, 3), padding='same', activation="relu"))
    model.add(MaxPool2D((2, 2)))

model.add(Flatten())
model.add(Dropout(DROPOUT_RATE))
model.add(Dense(FULLY_CONNECTED, activation="relu"))
model.add(Dropout(DROPOUT_RATE))
model.add(Dense(3, activation="softmax"))

In [28]:
model.compile('adam', 'categorical_crossentropy', metrics=['accuracy'])

In [29]:
model.layers[4].output.shape

TensorShape([None, 53, 9, 64])

In [30]:
# mock_X = np.random.randn(BATCH_SIZE * 10, FREQUENCIES, SAMPLES, CHANNELS)
# mock_y = np.eye(3)[np.random.choice(3, BATCH_SIZE * 10)]

In [32]:
model.fit(
    prepared_train_X, train_y, 
    validation_data=(prepared_test_X, test_y),
    epochs=10, batch_size=BATCH_SIZE
)

Train on 6396 samples, validate on 4845 samples
Epoch 1/10
6396/6396 [==============================] - 42s 7ms/step - loss: 0.6061 - accuracy: 0.7255 - val_loss: 0.3777 - val_accuracy: 0.7472
Epoch 2/10
6396/6396 [==============================] - 41s 6ms/step - loss: 0.3570 - accuracy: 0.7589 - val_loss: 0.3682 - val_accuracy: 0.7474
Epoch 3/10
6396/6396 [==============================] - 41s 6ms/step - loss: 0.3379 - accuracy: 0.7861 - val_loss: 0.3623 - val_accuracy: 0.7554
Epoch 4/10
6396/6396 [==============================] - 41s 6ms/step - loss: 0.3296 - accuracy: 0.8000 - val_loss: 0.3683 - val_accuracy: 0.7595
Epoch 5/10
6396/6396 [==============================] - 41s 6ms/step - loss: 0.3324 - accuracy: 0.8147 - val_loss: 0.3837 - val_accuracy: 0.7548
Epoch 6/10
6396/6396 [==============================] - 41s 6ms/step - loss: 0.3043 - accuracy: 0.8369 - val_loss: 0.3838 - val_accuracy: 0.7560
Epoch 7/10
1152/6396 [====>.........................] - ETA: 29s - loss: 0.2776 - 

KeyboardInterrupt: 

In [50]:
predictions = model.predict(prepared_train_X)
train_y_pred = to_categorical(np.argmax(predictions, axis=1).tolist())

In [53]:
from sklearn.metrics import accuracy_score, classification_report

In [55]:
print(classification_report(
    train_y, 
    train_y_pred
))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3181
           1       0.75      0.82      0.78      1608
           2       0.80      0.72      0.76      1607

   micro avg       0.89      0.89      0.89      6396
   macro avg       0.85      0.85      0.85      6396
weighted avg       0.89      0.89      0.88      6396
 samples avg       0.89      0.89      0.89      6396

